# Joining and merging

1. Basic merges
2. Join vs. merge
3. Column names
4. More sophisticated merges

In [3]:
import pandas as pd
from pandas import Series, DataFrame

# A little background/theory

Every data frame in Pandas is a 2D table. In theory, we could put *all* of the data that's related to our data set in a single data frame. In reality, the data will probably be broken up into different sources, and thus into different data frames.

This is even a good thing, in that it allows us to have many different data frames, each with its own speciality in terms of data. We can then "join" them together, as needed, when we need data from more than one place.

This is exactly how SQL (relational) databases work! In that world, it's considered a CRUCIAL thing to "normalize" your data, have it in a single place, and join tables together as needed.

When you join two tables together, you get one large, wide table. The joining is typically done on the index. So if you have one table with an index A, B, and C, and a second table with an index A, B, and C, you can think of joining as just taking the A row from both, the B row from both, and the C row from both, and combining them together.

When the index repeats, or when the index doesn't exist, things get a bit more interesting.

In [5]:
df = DataFrame([{'product_id':23, 'name':'computer',
                 'wholesale_price': 500,
                 'retail_price':1000, 'sales':100,
                 'department':'electronics'},
               {'product_id':96, 'name':'Python Workout',
                 'wholesale_price': 35,
                 'retail_price':75, 'sales':1000,
                 'department':'books'},
               {'product_id':97, 'name':'Pandas Workout',
                 'wholesale_price': 35,
                 'retail_price':75, 'sales':500,
                 'department':'books'},
               {'product_id':15, 'name':'banana',
                 'wholesale_price': 0.5,
                 'retail_price':1, 'sales':200,
                 'department':'food'},
               {'product_id':87, 'name':'sandwich',
                 'wholesale_price': 3,
                 'retail_price':5, 'sales':300,
                 'department': 'food'},
               ])

df

,product_id,name,wholesale_price,retail_price,sales,department
0,23,computer,500.0,1000,100,electronics
1,96,Python Workout,35.0,75,1000,books
2,97,Pandas Workout,35.0,75,500,books
3,15,banana,0.5,1,200,food
4,87,sandwich,3.0,5,300,food


# What's wrong with the above?

If we keep information about each product *and* sales in the same place.. what happens when we have a sale? How can we keep track of the customer's information? We'll add a new row for our product for every sale? 

The solution is to separate out this data frame, split it into two parts -- one will be for products, and another will be for sales. That allows us to do whatever we want with each data frame, and only join them together when we need data from both.

In [6]:
products_df = DataFrame([{'product_id':23, 'name':'computer',
                          'wholesale_price': 500,
                          'retail_price':1000,
                          'department':'electronics'},
                        {'product_id':96, 'name':'Python Workout',
                         'wholesale_price': 35,
                         'retail_price':75, 'department':'books'},
                        {'product_id':97, 'name':'Pandas Workout',
                         'wholesale_price': 35,
                         'retail_price':75, 'department':'books'},
                        {'product_id':15, 'name':'banana',
                         'wholesale_price': 0.5,
                         'retail_price':1, 'department':'food'},
                        {'product_id':87, 'name':'sandwich',
                        'wholesale_price': 3,
                         'retail_price':5, 'department': 'food'},
                        ])

sales_df = DataFrame([{'product_id': 23, 'date':'2021-August-10',
                       'quantity':1},
                     {'product_id': 96, 'date':'2021-August-10',
                       'quantity':5},
                     {'product_id': 15, 'date':'2021-August-10',
                       'quantity':3},
                     {'product_id': 87, 'date':'2021-August-10',
                       'quantity':2},
                     {'product_id': 15, 'date':'2021-August-11',
                       'quantity':1},
                     {'product_id': 96, 'date':'2021-August-11',
                       'quantity':1},
                     {'product_id': 23, 'date':'2021-August-11',
                       'quantity':2},
                     {'product_id': 87, 'date':'2021-August-12',
                       'quantity':2},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':6},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':1},
                     {'product_id': 87, 'date':'2021-August-13',
                       'quantity':2},
                     {'product_id': 23, 'date':'2021-August-13',
                       'quantity':1},
                     {'product_id': 15, 'date':'2021-August-14',
                       'quantity':2}
                     ])



In [7]:
products_df

,product_id,name,wholesale_price,retail_price,department
0,23,computer,500.0,1000,electronics
1,96,Python Workout,35.0,75,books
2,97,Pandas Workout,35.0,75,books
3,15,banana,0.5,1,food
4,87,sandwich,3.0,5,food


In [8]:
sales_df

,product_id,date,quantity
0,23,2021-August-10,1
1,96,2021-August-10,5
2,15,2021-August-10,3
3,87,2021-August-10,2
4,15,2021-August-11,1
5,96,2021-August-11,1
6,23,2021-August-11,2
7,87,2021-August-12,2
8,97,2021-August-12,6
9,97,2021-August-12,1


In [9]:
# if I just want to know about sales, I can ask sales_df.
# if I just want to know about products, I can ask products_df.

# What if I want to know how much revenue we got from sales?
# that is, given retail_price for each product, let's multiply that by the quantity?



# Joining

In Pandas, we can join together two data frames if their indexes match up. It's totally OK if one or more indexes repeats.

Here, we want to join `products_df` with `sales_df`. However, if we do that right now, we'll join them on the index that exists, which is *not* what we want!  We want to join them together using `product_id`.

So, I'll need to set the index in both data frames to use `product_id`, and then I can join them together.

In [14]:
(
    products_df
    .set_index('product_id')
    .join(sales_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
    ['revenue']
    .sum()
)

np.int64(5011)

In [16]:
# what if we were to join in the other direction?
# that is: start with sales_df and join with products_df?

(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,
23,2021-August-10,1,computer,500.0,1000,electronics,1000
96,2021-August-10,5,Python Workout,35.0,75,books,375
15,2021-August-10,3,banana,0.5,1,food,3
87,2021-August-10,2,sandwich,3.0,5,food,10
15,2021-August-11,1,banana,0.5,1,food,1
96,2021-August-11,1,Python Workout,35.0,75,books,75
23,2021-August-11,2,computer,500.0,1000,electronics,2000
87,2021-August-12,2,sandwich,3.0,5,food,10
97,2021-August-12,6,Pandas Workout,35.0,75,books,450


In [19]:
# let's remove two products from sales_df, and see what happens.

sales_df = (sales_df
            .loc[lambda df_: df_['product_id'].isin([23, 96, 15])]
           )
sales_df

,product_id,date,quantity
0,23,2021-August-10,1
1,96,2021-August-10,5
2,15,2021-August-10,3
4,15,2021-August-11,1
5,96,2021-August-11,1
6,23,2021-August-11,2
11,23,2021-August-13,1
12,15,2021-August-14,2


In [21]:
# what happens if I join products_df with sales_df?

(
    products_df
    .set_index('product_id')
    .join(sales_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,name,wholesale_price,retail_price,department,date,quantity,revenue
product_id,,,,,,,
23,computer,500.0,1000,electronics,2021-August-10,1.0,1000.0
23,computer,500.0,1000,electronics,2021-August-11,2.0,2000.0
23,computer,500.0,1000,electronics,2021-August-13,1.0,1000.0
96,Python Workout,35.0,75,books,2021-August-10,5.0,375.0
96,Python Workout,35.0,75,books,2021-August-11,1.0,75.0
97,Pandas Workout,35.0,75,books,NaN,NaN,NaN
15,banana,0.5,1,food,2021-August-10,3.0,3.0
15,banana,0.5,1,food,2021-August-11,1.0,1.0
15,banana,0.5,1,food,2021-August-14,2.0,2.0


In [22]:
# when we join products_df with sales_df, the left side determines what the index will be
# everything in products_df will appear in the final data frame, albeit with NaN values where there was no corresponding match

# what if I do it in the other direction?
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,
23,2021-August-10,1,computer,500.0,1000,electronics,1000
96,2021-August-10,5,Python Workout,35.0,75,books,375
15,2021-August-10,3,banana,0.5,1,food,3
15,2021-August-11,1,banana,0.5,1,food,1
96,2021-August-11,1,Python Workout,35.0,75,books,75
23,2021-August-11,2,computer,500.0,1000,electronics,2000
23,2021-August-13,1,computer,500.0,1000,electronics,1000
15,2021-August-14,2,banana,0.5,1,food,2


Because `sales_df` has no indexes 97 and 87, they won't appear in the final, joined
data frame. Which data frame is on the left, and thus "drives" the join, has a huge influence
on the final result.

This is known as a "left join," because the left side (i.e., the first data frame we mentioned)
dictates what the index will contain.



# `merge`

If this seems annoying to you, you're not alone!

Joining must take place on the index, so you're always going to check/set your index before running `join`. However, the `merge` method works on *any* columns. You can think of `join` as a specialized version of `merge`, in which a column on each is set to be the index.

I personally don't use `merge` much; I often set the index to join. But you don't have to!

In [26]:
pd.merge(products_df, sales_df, on='product_id')

,product_id,name,wholesale_price,retail_price,department,date,quantity
0,23,computer,500.0,1000,electronics,2021-August-10,1
1,23,computer,500.0,1000,electronics,2021-August-11,2
2,23,computer,500.0,1000,electronics,2021-August-13,1
3,96,Python Workout,35.0,75,books,2021-August-10,5
4,96,Python Workout,35.0,75,books,2021-August-11,1
5,15,banana,0.5,1,food,2021-August-10,3
6,15,banana,0.5,1,food,2021-August-11,1
7,15,banana,0.5,1,food,2021-August-14,2


In [25]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'MergeHow' = 'inner',
    on: 'IndexLabel | AnyArrayLike | None' = None,
    left_on: 'IndexLabel | AnyArrayLike | None' = None,
    right_on: 'IndexLabel | AnyArrayLike | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool | None' = None,
    indicator: 'str | bool' = False,
    validate: 'str | None' = None
) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.

    A named Series object is treated as a DataFrame with a single named column.

    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no 

In [27]:
products_df

,product_id,name,wholesale_price,retail_price,department
0,23,computer,500.0,1000,electronics
1,96,Python Workout,35.0,75,books
2,97,Pandas Workout,35.0,75,books
3,15,banana,0.5,1,food
4,87,sandwich,3.0,5,food


In [28]:
sales_df

,product_id,date,quantity
0,23,2021-August-10,1
1,96,2021-August-10,5
2,15,2021-August-10,3
4,15,2021-August-11,1
5,96,2021-August-11,1
6,23,2021-August-11,2
11,23,2021-August-13,1
12,15,2021-August-14,2


In [29]:
sales_df = DataFrame([{'product_id': 23, 'date':'2021-August-10',
                       'quantity':1, 'customer_id':1},
                     {'product_id': 96, 'date':'2021-August-10',
                       'quantity':5, 'customer_id':1},
                     {'product_id': 15, 'date':'2021-August-10',
                       'quantity':3, 'customer_id':1},
                     {'product_id': 87, 'date':'2021-August-10',
                       'quantity':2, 'customer_id':1},
                     {'product_id': 15, 'date':'2021-August-11',
                       'quantity':1, 'customer_id':1},
                     {'product_id': 96, 'date':'2021-August-11',
                       'quantity':1, 'customer_id':2},
                     {'product_id': 23, 'date':'2021-August-11',
                       'quantity':2, 'customer_id':2},
                     {'product_id': 87, 'date':'2021-August-12',
                       'quantity':2, 'customer_id':2},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':6, 'customer_id':2},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':1, 'customer_id':2},
                     {'product_id': 87, 'date':'2021-August-13',
                       'quantity':2, 'customer_id':2},
                     {'product_id': 23, 'date':'2021-August-13',
                       'quantity':1, 'customer_id':2},
                     {'product_id': 15, 'date':'2021-August-14',
                       'quantity':2, 'customer_id':2}
                     ])

customers_df = DataFrame([{'customer_id':1,
                           'name':'Reuven'},
                          {'customer_id':2,
                           'name':'Whoever'}])

In [30]:
pd.merge(products_df, sales_df, on='product_id')

,product_id,name,wholesale_price,retail_price,department,date,quantity,customer_id
0,23,computer,500.0,1000,electronics,2021-August-10,1,1
1,23,computer,500.0,1000,electronics,2021-August-11,2,2
2,23,computer,500.0,1000,electronics,2021-August-13,1,2
3,96,Python Workout,35.0,75,books,2021-August-10,5,1
4,96,Python Workout,35.0,75,books,2021-August-11,1,2
5,97,Pandas Workout,35.0,75,books,2021-August-12,6,2
6,97,Pandas Workout,35.0,75,books,2021-August-12,1,2
7,15,banana,0.5,1,food,2021-August-10,3,1
8,15,banana,0.5,1,food,2021-August-11,1,1
9,15,banana,0.5,1,food,2021-August-14,2,2


In [35]:
# now let's try to do a 3-way merge!

(
    products_df
    .set_index('product_id')
    .join(sales_df.set_index('product_id'))
    .reset_index()
    .set_index('customer_id')
    .join(customers_df.set_index('customer_id'))
)

ValueError: columns overlap but no suffix specified: Index(['name'], dtype='object')

# Unique columns!

In Pandas, the index (i.e., row labels) don't need to be unique. But the columns *always* have to have unique names. Here, the result of our join would be two `name` columns -- one from `products_df` and another from `customers_df`.

The solution is to provide Pandas with a suffix for the left data frame, the right data frame, or both!

In [38]:
(
    products_df
    .set_index('product_id')
    .join(sales_df.set_index('product_id'))
    .reset_index()
    .set_index('customer_id')
    .join(customers_df
          .set_index('customer_id'),
          lsuffix='_products',
          rsuffix='_customers')
          
)

,product_id,name_products,wholesale_price,retail_price,department,date,quantity,name_customers
customer_id,,,,,,,,
1,23,computer,500.0,1000,electronics,2021-August-10,1,Reuven
2,23,computer,500.0,1000,electronics,2021-August-11,2,Whoever
2,23,computer,500.0,1000,electronics,2021-August-13,1,Whoever
1,96,Python Workout,35.0,75,books,2021-August-10,5,Reuven
2,96,Python Workout,35.0,75,books,2021-August-11,1,Whoever
2,97,Pandas Workout,35.0,75,books,2021-August-12,6,Whoever
2,97,Pandas Workout,35.0,75,books,2021-August-12,1,Whoever
1,15,banana,0.5,1,food,2021-August-10,3,Reuven
1,15,banana,0.5,1,food,2021-August-11,1,Reuven


# More interesting/sophisticated joins

We've seen that if we join `products_df` and `sales_df`, the direction matters! The left side determines which index will be used.

Sometimes, we can choose the direction. But sometimes, we can't!  In such cases, we can still tell Pandas that even though we're saying `A.join(B)`, it should handle things as if we did `B.join(A)`. We call this a *right* join.

In [40]:
# remove some rows from sales_df
sales_df = (sales_df
            .loc[lambda df_: df_['product_id'].isin([23, 96, 15])]
           )
sales_df

# perform the join with sales_df on the left
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
23,2021-August-10,1,1,computer,500.0,1000,electronics,1000
96,2021-August-10,5,1,Python Workout,35.0,75,books,375
15,2021-August-10,3,1,banana,0.5,1,food,3
15,2021-August-11,1,1,banana,0.5,1,food,1
96,2021-August-11,1,2,Python Workout,35.0,75,books,75
23,2021-August-11,2,2,computer,500.0,1000,electronics,2000
23,2021-August-13,1,2,computer,500.0,1000,electronics,1000
15,2021-August-14,2,2,banana,0.5,1,food,2


In [41]:
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'),
         how='right')   # this means: use the right side to determine the index, rather than the left side
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
23,2021-August-10,1.0,1.0,computer,500.0,1000,electronics,1000.0
23,2021-August-11,2.0,2.0,computer,500.0,1000,electronics,2000.0
23,2021-August-13,1.0,2.0,computer,500.0,1000,electronics,1000.0
96,2021-August-10,5.0,1.0,Python Workout,35.0,75,books,375.0
96,2021-August-11,1.0,2.0,Python Workout,35.0,75,books,75.0
97,NaN,NaN,NaN,Pandas Workout,35.0,75,books,NaN
15,2021-August-10,3.0,1.0,banana,0.5,1,food,3.0
15,2021-August-11,1.0,1.0,banana,0.5,1,food,1.0
15,2021-August-14,2.0,2.0,banana,0.5,1,food,2.0


In [42]:
# we now have some items in products_df that aren't in sales_df.
# what if we have some rows in sales_df that aren't in products_df?  (Meaning, we're referencing products that don't exist?)

new_sales_df = DataFrame([{'product_id': 999, 'date':'2024-October-27',
                       'quantity':3, 'customer_id':2},
                        {'product_id': 998, 'date':'2024-October-27',
                       'quantity':3, 'customer_id':1},
                        
                     ])

In [45]:
# add some new stuff to sales_df
sales_df = pd.concat([sales_df, new_sales_df])

In [47]:
# what happens if we join on sales_df?
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'))
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
23,2021-August-10,1,1,computer,500.0,1000.0,electronics,1000.0
96,2021-August-10,5,1,Python Workout,35.0,75.0,books,375.0
15,2021-August-10,3,1,banana,0.5,1.0,food,3.0
15,2021-August-11,1,1,banana,0.5,1.0,food,1.0
96,2021-August-11,1,2,Python Workout,35.0,75.0,books,75.0
23,2021-August-11,2,2,computer,500.0,1000.0,electronics,2000.0
23,2021-August-13,1,2,computer,500.0,1000.0,electronics,1000.0
15,2021-August-14,2,2,banana,0.5,1.0,food,2.0
999,2024-October-27,3,2,NaN,NaN,NaN,NaN,NaN


In [48]:
# if we reverse it with how='right'...
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'), how='right')
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
23,2021-August-10,1.0,1.0,computer,500.0,1000,electronics,1000.0
23,2021-August-11,2.0,2.0,computer,500.0,1000,electronics,2000.0
23,2021-August-13,1.0,2.0,computer,500.0,1000,electronics,1000.0
96,2021-August-10,5.0,1.0,Python Workout,35.0,75,books,375.0
96,2021-August-11,1.0,2.0,Python Workout,35.0,75,books,75.0
97,NaN,NaN,NaN,Pandas Workout,35.0,75,books,NaN
15,2021-August-10,3.0,1.0,banana,0.5,1,food,3.0
15,2021-August-11,1.0,1.0,banana,0.5,1,food,1.0
15,2021-August-14,2.0,2.0,banana,0.5,1,food,2.0


In [49]:
# what if we want all of the rows from both data frames?
# we call that an outer join -- where both the left and right get to provide rows, even if there is no 
# corresponding row in the other data frame

(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'), how='outer')
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
15,2021-August-10,3.0,1.0,banana,0.5,1.0,food,3.0
15,2021-August-11,1.0,1.0,banana,0.5,1.0,food,1.0
15,2021-August-14,2.0,2.0,banana,0.5,1.0,food,2.0
23,2021-August-10,1.0,1.0,computer,500.0,1000.0,electronics,1000.0
23,2021-August-11,2.0,2.0,computer,500.0,1000.0,electronics,2000.0
23,2021-August-13,1.0,2.0,computer,500.0,1000.0,electronics,1000.0
87,NaN,NaN,NaN,sandwich,3.0,5.0,food,NaN
96,2021-August-10,5.0,1.0,Python Workout,35.0,75.0,books,375.0
96,2021-August-11,1.0,2.0,Python Workout,35.0,75.0,books,75.0


In [50]:
help(sales_df.join)

Help on method join in module pandas.core.frame:

join(
    other: 'DataFrame | Series | Iterable[DataFrame | Series]',
    on: 'IndexLabel | None' = None,
    how: 'MergeHow' = 'left',
    lsuffix: 'str' = '',
    rsuffix: 'str' = '',
    sort: 'bool' = False,
    validate: 'JoinValidate | None' = None
) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Join columns of another DataFrame.

    Join columns with `other` DataFrame either on index or on a key
    column. Efficiently join multiple DataFrame objects by index at once by
    passing a list.

    Parameters
    ----------
    other : DataFrame, Series, or a list containing any combination of them
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame.
    on : str, list of str, or array-like, optional
        Column or index level name(s) in the caller to 

In [51]:
(
    sales_df
    .set_index('product_id')
    .join(products_df.set_index('product_id'), how='inner')
    .assign(revenue=lambda df_: df_['retail_price'] * df_['quantity'])
)

,date,quantity,customer_id,name,wholesale_price,retail_price,department,revenue
product_id,,,,,,,,
23,2021-August-10,1,1,computer,500.0,1000,electronics,1000
96,2021-August-10,5,1,Python Workout,35.0,75,books,375
15,2021-August-10,3,1,banana,0.5,1,food,3
15,2021-August-11,1,1,banana,0.5,1,food,1
96,2021-August-11,1,2,Python Workout,35.0,75,books,75
23,2021-August-11,2,2,computer,500.0,1000,electronics,2000
23,2021-August-13,1,2,computer,500.0,1000,electronics,1000
15,2021-August-14,2,2,banana,0.5,1,food,2
